In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import log_loss
import seaborn as sns
import sklearn.preprocessing as pp
from scipy import optimize
from sklearn.metrics import log_loss

In [2]:
path = "../data/09_oof/"
knn2 = pd.read_csv(path + "knn_2.csv").iloc[:,1:].values
knn4 = pd.read_csv(path + "knn_4.csv").iloc[:,1:].values
knn8 = pd.read_csv(path + "knn_8.csv").iloc[:,1:].values
knn16 = pd.read_csv(path + "knn_16.csv").iloc[:,1:].values
knn32 = pd.read_csv(path + "knn_32.csv").iloc[:,1:].values
knn64 = pd.read_csv(path + "knn_64.csv").iloc[:,1:].values
knn128 = pd.read_csv(path + "knn_128.csv").iloc[:,1:].values
knn256 = pd.read_csv(path + "knn_256.csv").iloc[:,1:].values
knn512 = pd.read_csv(path + "knn_512.csv").iloc[:,1:].values
knn1024 = pd.read_csv(path + "knn_1024.csv").iloc[:,1:].values
lgbm1 = pd.read_csv(path + "lgbm_normal.csv").iloc[:, 1:].values
lgbm2 = pd.read_csv(path + "lgbm_normal_2.csv").iloc[:, 1:].values

nn1 = pd.read_csv(path + "nn_normal.csv").iloc[:, 1:].values
nn2 = pd.read_csv(path + "nn_normal_2.csv").iloc[:, 1:].values
nn3 = pd.read_csv(path + "nn_normal_3.csv").iloc[:, 1:].values
nn4 = pd.read_csv(path + "nn_normal_4.csv").iloc[:, 1:].values
nn5 = pd.read_csv(path + "nn_normal_5.csv").iloc[:, 1:].values


et1 = pd.read_csv(path + "extra_1.csv").iloc[:, 1:].values
et2 = pd.read_csv(path + "extra_2.csv").iloc[:, 1:].values
et3 = pd.read_csv(path + "extra_3.csv").iloc[:, 1:].values
rgf1 = pd.read_csv(path + "rgf_1.csv").iloc[:, 1:].values
rgf2 = pd.read_csv(path + "rgf_2.csv").iloc[:, 1:].values
rgf3 = pd.read_csv(path + "rgf_3.csv").iloc[:, 1:].values

In [3]:
assert knn2.shape == lgbm1.shape == nn1.shape
knn2.shape, lgbm1.shape, nn1.shape

((206246, 9), (206246, 9), (206246, 9))

In [4]:
train = pd.read_csv("../data/04_features/train2_csv")
target = pd.read_csv("../data/01_raw/train.csv")["target"]
train

,1,2,3,4,5,6,7,8,9,10,...,85,86,87,88,89,90,91,92,93,count_zero
0,0.693147,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.693147,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,70
1,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.693147,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,83
2,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.693147,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,85
3,0.693147,0.000000,0.0,0.693147,1.94591,0.693147,1.791759,0.000000,0.0,0.693147,...,0.000000,0.693147,1.098612,0.000000,0.0,0.000000,0.000000,0.000000,0.0,55
4,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.693147,0.000000,0.000000,0.000000,0.0,0.693147,0.000000,0.000000,0.0,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206241,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.693147,...,0.000000,1.098612,0.693147,0.693147,0.0,0.000000,0.000000,0.000000,0.0,75
206242,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.693147,1.609438,0.693147,2.484907,0.0,0.000000,0.000000,0.000000,0.0,84
206243,0.000000,0.693147,0.0,0.000000,0.00000,0.000000,0.693147,0.693147,0.0,0.000000,...,0.693147,1.386294,0.693147,0.693147,0.0,0.000000,0.693147,0.000000,0.0,64
206244,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,1.791759,0.0,0.000000,0.000000,0.693147,0.0,82


In [5]:
target

0        Class_1
1        Class_1
2        Class_1
3        Class_1
4        Class_1
          ...   
61873    Class_9
61874    Class_9
61875    Class_9
61876    Class_9
61877    Class_9
Name: target, Length: 61878, dtype: object

In [6]:
dict_a = {"Class_1": 0,
          "Class_2": 1,
          "Class_3": 2,
          "Class_4": 3,
          "Class_5": 4,
          "Class_6": 5,
          "Class_7": 6,
          "Class_8": 7,
          "Class_9": 8,
          }
target = target.map(dict_a)
target

0        0
1        0
2        0
3        0
4        0
        ..
61873    8
61874    8
61875    8
61876    8
61877    8
Name: target, Length: 61878, dtype: int64

In [7]:
df_train = pd.concat([train[:len(target)].reset_index(drop=True), 
                      pd.DataFrame(knn2[:len(target)]),
                      pd.DataFrame(knn4[:len(target)]),
                      pd.DataFrame(knn8[:len(target)]),
                      pd.DataFrame(knn16[:len(target)]),
                      pd.DataFrame(knn32[:len(target)]),
                      pd.DataFrame(knn64[:len(target)]),
                      pd.DataFrame(knn128[:len(target)]),
                      pd.DataFrame(knn256[:len(target)]),
                      pd.DataFrame(knn512[:len(target)]),
                      pd.DataFrame(knn1024[:len(target)]),
                      pd.DataFrame(lgbm1[:len(target)]),
                      pd.DataFrame(lgbm2[:len(target)]),
                      pd.DataFrame(nn1[:len(target)]),
                      pd.DataFrame(nn2[:len(target)]),
                      pd.DataFrame(nn3[:len(target)]),
                      pd.DataFrame(nn4[:len(target)]),
                      pd.DataFrame(nn5[:len(target)]),
                      pd.DataFrame(et1[:len(target)]),
                      pd.DataFrame(et2[:len(target)]),
                      pd.DataFrame(et3[:len(target)]),
                      pd.DataFrame(rgf1[:len(target)]),
                      pd.DataFrame(rgf2[:len(target)]),
                      pd.DataFrame(rgf3[:len(target)]),
                     ], axis=1)

In [8]:
df_train

,1,2,3,4,5,6,7,8,9,10,...,8,0,1,2,3,4,5,6,7,8
0,0.693147,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005464,0.011098,0.005954,0.003352,0.004700,0.001364,0.948844,0.007017,0.002857,0.014814
1,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,...,0.019690,0.140834,0.057571,0.008359,0.005280,0.007085,0.064974,0.120046,0.566535,0.029316
2,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,...,0.303656,0.572246,0.013105,0.006886,0.004154,0.009718,0.055987,0.035922,0.034527,0.267455
3,0.693147,0.0,0.0,0.693147,1.945910,0.693147,1.791759,0.000000,0.000000,0.693147,...,0.013138,0.508740,0.010811,0.029406,0.006563,0.001199,0.061776,0.059526,0.254195,0.067783
4,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.036283,0.323841,0.024379,0.009487,0.003023,0.004489,0.049016,0.134797,0.421631,0.029337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61873,0.693147,0.0,0.0,0.693147,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.740218,0.026144,0.045864,0.005121,0.014455,0.024098,0.053935,0.004096,0.047778,0.778509
61874,1.609438,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.713409,0.045887,0.048746,0.028111,0.031757,0.002676,0.026790,0.004739,0.029989,0.781304
61875,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.386294,0.693147,0.000000,...,0.923105,0.058491,0.013752,0.009055,0.002416,0.001917,0.019079,0.003731,0.007709,0.883851
61876,0.693147,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.873914,0.053718,0.002743,0.001735,0.003451,0.001991,0.159108,0.004699,0.021432,0.751122


In [9]:
df_test = pd.concat([train[len(target):].reset_index(drop=True), 
                      pd.DataFrame(knn2[len(target):]),
                      pd.DataFrame(knn4[len(target):]),
                      pd.DataFrame(knn8[len(target):]),
                      pd.DataFrame(knn16[len(target):]),
                      pd.DataFrame(knn32[len(target):]),
                      pd.DataFrame(knn64[len(target):]),
                      pd.DataFrame(knn128[len(target):]),
                      pd.DataFrame(knn256[len(target):]),
                      pd.DataFrame(knn512[len(target):]),
                      pd.DataFrame(knn1024[len(target):]),
                      pd.DataFrame(lgbm1[len(target):]),
                     pd.DataFrame(lgbm2[len(target):]),
                      pd.DataFrame(nn1[len(target):]),
                     pd.DataFrame(nn2[len(target):]),
                     pd.DataFrame(nn3[len(target):]),
                     pd.DataFrame(nn4[len(target):]),
                     pd.DataFrame(nn5[len(target):]),
                     pd.DataFrame(et1[len(target):]),
                     pd.DataFrame(et2[len(target):]),
                     pd.DataFrame(et3[len(target):]),
                     pd.DataFrame(rgf1[len(target):]),
                     pd.DataFrame(rgf2[len(target):]),
                     pd.DataFrame(rgf2[len(target):]),
                     ], axis=1)

In [10]:
df_test

,1,2,3,4,5,6,7,8,9,10,...,8,0,1,2,3,4,5,6,7,8
0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.386294,...,0.001541,0.006329,0.239421,0.295196,0.414912,0.001945,0.006692,0.032452,0.001511,0.001541
1,1.098612,1.098612,2.708050,2.833213,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.005161,0.012184,0.029519,0.006074,0.008425,0.001809,0.813108,0.004849,0.118871,0.005161
2,0.000000,0.693147,2.564949,0.693147,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.002198,0.001234,0.006457,0.002832,0.005635,0.001485,0.970997,0.001829,0.007333,0.002198
3,0.000000,0.000000,0.000000,0.693147,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.124879,0.005903,0.385606,0.432265,0.042891,0.001617,0.002053,0.001854,0.002931,0.124879
4,0.693147,0.000000,0.000000,0.693147,0.0,0.0,0.693147,1.098612,0.0,1.386294,...,0.584350,0.131318,0.002228,0.001880,0.001348,0.002446,0.015081,0.004310,0.257040,0.584350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144363,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.693147,...,0.092773,0.631164,0.007132,0.003610,0.006844,0.001968,0.146027,0.085652,0.024830,0.092773
144364,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.001809,0.005243,0.345937,0.444928,0.107252,0.003967,0.002057,0.087875,0.000932,0.001809
144365,0.000000,0.693147,0.000000,0.000000,0.0,0.0,0.693147,0.693147,0.0,0.000000,...,0.000867,0.004561,0.493569,0.314225,0.159159,0.001779,0.004095,0.017570,0.004174,0.000867
144366,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000996,0.001940,0.604973,0.058417,0.323900,0.003992,0.000908,0.003705,0.001168,0.000996


In [20]:
params = {
    'bagging_freq': 5,          'bagging_fraction': 0.8,   'boost_from_average':'false',   'boost': 'gbdt',
    'feature_fraction': 0.3,   'learning_rate': 0.04,     'max_depth': -1,  'metrics':'multi_logloss',   
    'min_data_in_leaf': 10,     'min_sum_hessian_in_leaf': 8.0,'num_leaves': 8,           'num_threads': 8,
    'tree_learner': 'serial',   'objective': 'multiclass',   'num_class': 9,   'verbosity': 1
}

In [12]:
NFOLDS = 5
RANDOM_STATE = 871975

excluded_column = ['target', 'id']
cols = [c for c in train.columns if c not in (excluded_column + [])]

folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, 
                        random_state=RANDOM_STATE)

In [21]:
y_pred_lgb = np.zeros((len(df_test), 9))
oof = np.zeros((len(train), 9))
score = 0
feature_importance_df = pd.DataFrame()
valid_predict = []
for fold_n, (train_index, valid_index) in enumerate(folds.split(df_train, target)):
    print('Fold', fold_n)
    X_train, X_valid = df_train.iloc[train_index], df_train.iloc[valid_index]
    y_train, y_valid = target[train_index].astype(int), target[valid_index].astype(int)
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)

    best_params = {}
    tuning_history = []

    lgb_model = lgb.train(params,train_data,num_boost_round=30000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 300)
    
    y_valid_predict = lgb_model.predict(X_valid, num_iteration=lgb_model.best_iteration)
    score += log_loss(y_valid, y_valid_predict)
    oof[valid_index] = y_valid_predict
    print('Fold', fold_n, 'valid logloss', log_loss(y_valid, y_valid_predict))
    
    y_pred_lgb += lgb_model.predict(df_test, num_iteration=lgb_model.best_iteration)/NFOLDS
    
print('valid logloss average:', score/NFOLDS)

Fold 0
Training until validation scores don't improve for 300 rounds
[300]	training's multi_logloss: 0.360868	valid_1's multi_logloss: 0.420626
[600]	training's multi_logloss: 0.316406	valid_1's multi_logloss: 0.422414
Early stopping, best iteration is:
[301]	training's multi_logloss: 0.360705	valid_1's multi_logloss: 0.420584
Fold 0 valid logloss 0.4205835147388894
Fold 1
Training until validation scores don't improve for 300 rounds
[300]	training's multi_logloss: 0.361841	valid_1's multi_logloss: 0.416482
[600]	training's multi_logloss: 0.3167	valid_1's multi_logloss: 0.416799
Early stopping, best iteration is:
[390]	training's multi_logloss: 0.346808	valid_1's multi_logloss: 0.41596
Fold 1 valid logloss 0.41596000887537776
Fold 2
Training until validation scores don't improve for 300 rounds
[300]	training's multi_logloss: 0.363121	valid_1's multi_logloss: 0.41277
[600]	training's multi_logloss: 0.317931	valid_1's multi_logloss: 0.413668
Early stopping, best iteration is:
[309]	train

In [14]:
y_pred_lgb.shape

(144368, 9)

In [15]:
ss = pd.read_csv("../data/01_raw/sampleSubmission.csv")

In [22]:
ss.iloc[:, 1:] = y_pred_lgb
ss

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,0.000276,0.126747,0.150949,0.716589,0.000098,0.000437,0.004402,0.000257,0.000244
1,2,0.000308,0.000405,0.000531,0.000727,0.000100,0.144915,0.000491,0.851660,0.000864
2,3,0.000036,0.000027,0.000030,0.000045,0.000007,0.999728,0.000039,0.000065,0.000024
3,4,0.000338,0.569182,0.415067,0.013680,0.000148,0.000368,0.000479,0.000350,0.000388
4,5,0.109144,0.000531,0.000874,0.000438,0.000117,0.004165,0.001440,0.017552,0.865739
...,...,...,...,...,...,...,...,...,...,...
144363,144364,0.386847,0.001058,0.000866,0.004563,0.000204,0.395600,0.063074,0.058601,0.089186
144364,144365,0.000410,0.362837,0.535082,0.059701,0.000134,0.000516,0.040637,0.000363,0.000319
144365,144366,0.000499,0.428321,0.375792,0.177809,0.000148,0.000672,0.016030,0.000389,0.000340
144366,144367,0.000244,0.375978,0.021335,0.600462,0.000154,0.000351,0.001020,0.000249,0.000208


In [23]:
ss.to_csv("../data/07_model_output/stacking13.csv", index=None)